In [1]:
import geopandas as gpd
import pandas as pd
from copy import deepcopy
from geopandas import GeoDataFrame
from shapely.ops import unary_union

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
gdf_center = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/国土数値情報/土地利用メッシュ/gdf_center.shp')

In [32]:
gdf_railway = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/国土数値情報/鉄道/N02-22_RailroadSection.shp', encoding='shift-jis') #このデータは道路中心線

In [33]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_center.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_center = gdf_center.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [34]:
#座標系を変更
src_proj = 6668 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_railway.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_railway = gdf_railway.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [41]:
result

,Meshcode,longitude,latitude,geometry_x,index_road,Railway_classification,Railway_Business_Type,Railway_route_name,Railway_operating_company,nearest_railway
0,5840378485,140.931875,38.990417,POINT (8536.609 -112073.115),16468,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -..."
1,5840378486,140.933125,38.990417,POINT (8644.896 -112072.997),16468,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -..."
2,5840378487,140.934375,38.990417,POINT (8753.182 -112072.878),16468,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -..."
3,5840378495,140.931875,38.991250,POINT (8536.509 -111980.612),16468,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -..."
4,5840378496,140.933125,38.991250,POINT (8644.794 -111980.494),16468,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -..."
...,...,...,...,...,...,...,...,...,...,...
101746,5841726411,141.301875,39.301250,POINT (40411.455 -77468.347),14088,11,2,釜石線,東日本旅客鉄道,"LINESTRING (47135.057 -76857.784, 47136.594 -7..."
101747,5841726412,141.303125,39.301250,POINT (40519.267 -77467.788),14088,11,2,釜石線,東日本旅客鉄道,"LINESTRING (47135.057 -76857.784, 47136.594 -7..."
101748,5841726421,141.301875,39.302083,POINT (40410.976 -77375.838),14088,11,2,釜石線,東日本旅客鉄道,"LINESTRING (47135.057 -76857.784, 47136.594 -7..."
101749,5841726422,141.303125,39.302083,POINT (40518.787 -77375.279),14088,11,2,釜石線,東日本旅客鉄道,"LINESTRING (47135.057 -76857.784, 47136.594 -7..."


In [35]:
#線路のデータフレームのカラム名を変換
gdf_railway.rename(columns={'N02_001': 'Railway_classification', 'N02_002': 'Railway_Business_Type', 'N02_003': 'Railway_route_name', 'N02_004':'Railway_operating_company'}, inplace=True)

In [36]:
#空間結合した後にgeomeryを加えられるように連番を追加
number = [i for i in range(len(gdf_railway))]
gdf_railway['number'] = number

In [40]:
#空間結合をする
result = gpd.sjoin_nearest(gdf_center, gdf_railway, how="left",lsuffix='point',rsuffix='road')
#空間結合された道路のgeometryを追加
result = pd.merge(result, gdf_railway[['number', 'geometry']], on='number')
#geometryの名前を変更
result.rename(columns={'geometry_y':'nearest_railway'}, inplace=True)
#メモリの解放
# del gdf_railway

result.drop('number', axis=1, inplace=True)

In [42]:
#dictance_railwyカラムを追加
result['distance_railway'] = None

for i in range((len(result) // 50)+1):
    try:
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_railway'][50*i:50*(i+1)]))

        # ポイントとマルチラインストリングとの最短距離を計算
        result['distance_railway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
        #一回ごとにメモリを解放
        del merged_lines
    except Exception as e:
        # エラーが発生した場合の処理
        print(f"Error occurred at the last batch: {e}")
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_railway'][50*(i+1):]))

        result['distance_railway'][50*(i+1):] = result.geometry_x[50*(i+1):].apply(lambda p: p.distance(merged_lines))

ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-42-2e00476379e1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_railway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-42-2e00476379e1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_railway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-42-2e00476379e1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [43]:
#重複を削除
result = result[~result.Meshcode.duplicated()]

#indexを並び替え
result.reset_index(inplace=True)

In [44]:
result_columns = result.columns.to_list()[6:]
result_columns[:0] = ['Meshcode']
result[result_columns].to_csv('/content/drive/Shareddrives/geomap/周辺環境データ構築/dev/奥州市/奥州市　最近傍鉄道/奥州市_最近傍線路.csv', encoding='shift-jis', index=False)

In [48]:
result[result_columns]

,Meshcode,Railway_classification,Railway_Business_Type,Railway_route_name,Railway_operating_company,nearest_railway,distance_railway
0,5840378485,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -...",15423.440836
1,5840378486,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -...",15315.403148
2,5840378487,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -...",15207.368899
3,5840378495,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -...",15417.466519
4,5840378496,11,2,東北線,東日本旅客鉄道,"LINESTRING (24608.198 -112203.271, 24553.137 -...",15309.387942
...,...,...,...,...,...,...,...
101381,5841726411,11,2,釜石線,東日本旅客鉄道,"LINESTRING (47135.057 -76857.784, 47136.594 -7...",5958.732188
101382,5841726412,11,2,釜石線,東日本旅客鉄道,"LINESTRING (47135.057 -76857.784, 47136.594 -7...",5879.246268
101383,5841726421,11,2,釜石線,東日本旅客鉄道,"LINESTRING (47135.057 -76857.784, 47136.594 -7...",5897.048421
101384,5841726422,11,2,釜石線,東日本旅客鉄道,"LINESTRING (47135.057 -76857.784, 47136.594 -7...",5816.720601
